In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import terminalLogic
from terminalLogic import Algo
import client
from client import engine
import pandas as pd
import numpy as np
import datetime
import time
import seaborn as sb
plt.rcParams['figure.figsize'] = [16.0, 6.0]



In [2]:
url = engine()
data = url.getData()
df = data
df

,time,open,high,low,close,volume
0,2021-11-02 15:09:00,63514.0,63538.0,63505.0,63521.0,5.262254e+05
1,2021-11-02 15:10:00,63521.0,63533.0,63480.0,63488.0,2.464984e+06
2,2021-11-02 15:11:00,63488.0,63505.0,63465.0,63491.0,8.007969e+05
3,2021-11-02 15:12:00,63491.0,63516.0,63470.0,63516.0,1.391909e+06
4,2021-11-02 15:13:00,63516.0,63516.0,63483.0,63494.0,4.392092e+05
...,...,...,...,...,...,...
1495,2021-11-03 16:04:00,62261.0,62346.0,62245.0,62345.0,8.932825e+05
1496,2021-11-03 16:05:00,62345.0,62381.0,62301.0,62342.0,1.473654e+06
1497,2021-11-03 16:06:00,62342.0,62342.0,62298.0,62298.0,8.707808e+05
1498,2021-11-03 16:07:00,62298.0,62340.0,62298.0,62338.0,7.946052e+05


In [4]:
df1 = Algo.fullstate(df, df)
df1.dropna(inplace=True)
df2 = df1
df2['time'] = df2['time'].dt.date
table = df2.tail(20)
table.to_json('./assets/table.json', orient='records')


AttributeError: Can only use .dt accessor with datetimelike values

## Trend positions

In [ ]:
fig = plt.figure(facecolor=(1, 1, 1))
y = df2.iloc[-500:]['time']
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(df2.iloc[-500:]['close'], label = 'BTC')
plt.plot(df2.iloc[-500:]['9-min'], label = '9-min')
plt.plot(df2.iloc[-500:]['21-min'], label = '21-min')
plt.plot(df2[-500:].loc[df2.entry == 2].index, df2[-500:]['9-min'][df2.entry == 2], "^",
        color = "g", markersize = 12, label= "Long")
plt.plot(df2[-500:].loc[df2.entry == -2].index, df2[-500:]['21-min'][df2.entry == -2], "v",
        color = "r", markersize = 12, label="Short")
plt.legend(loc=2);
plt.savefig('./assets/trendPositions.png')

# Trend Algo Performance

In [ ]:
fig = plt.figure(facecolor=(1, 1, 1))
y=df2.iloc[-500:]['time']
df2['trend_returns'] = df2.signal * df2.market_returns
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(np.exp(df2.iloc[-500:]['market_returns']).cumprod(),label = "Buy/Hold")
plt.plot(np.exp(df2.iloc[-500:]['trend_returns']).cumprod(),label = "Strat")
plt.legend()
plt.savefig('./assets/trendRets.png')

In [ ]:
fig = plt.figure(facecolor=(1, 1, 1))
y=df2.iloc[-500:]['time']
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(df2.iloc[-500:]['close'], label = 'BTC')
plt.plot(df2.iloc[-500:]['9-min'], label = '9-min')
plt.legend(loc=2);
plt.savefig('./assets/btc1m9ma.png')

In [ ]:
sb.set()
y = df2.iloc[-500:]['time']
percentiles = [5, 10, 50, 90, 95]
p = np.percentile(df2['ratio'].dropna(), percentiles)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(df2.iloc[-500:]['ratio'].dropna())
plt.axhline(p[0], c=(.5, .5, .5), ls='--')
plt.axhline(p[2], c=(.5, .5, .5), ls='--')
plt.axhline(p[-1], c=(.5, .5, .5), ls='--')
plt.savefig('./assets/rangePercentiles.png')

# Range positions

In [ ]:
plt.rcParams['figure.figsize'] = [16.0, 6.0]

fig = plt.figure(facecolor=(1, 1, 1))
y=df2.iloc[-500:]['time']
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(df2.iloc[-500:]['position'].dropna())
plt.savefig('./assets/rangeStatus.png')


In [ ]:
""" Plots returns for the range algo df thus returns 'R' """
y=df2.iloc[-500:]['time'].dropna()
fig = plt.figure(facecolor=(1, 1, 1))
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(np.exp(df2.iloc[-500:]['market_returns'].dropna()).cumprod(), label='Buy/Hold')
plt.plot(np.exp(df2.iloc[-500:]['range_returns'].dropna()).cumprod(), label='Strategy')
plt.legend();
plt.savefig('.//assets/rangeRets.png')

In [ ]:
fig = plt.figure(facecolor=(1, 1, 1))

y=df2.iloc[-500:]['time']
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.plot(df2.iloc[-500:]['close'], label = 'BTC')
plt.plot(df2.iloc[-500:]['9-min'], label = '9-min')
plt.plot(df2[-500:].loc[df2.entryR == 2].index, df2[-500:]['9-min'][df2.entryR == 2], "^",
        color = "r", markersize = 12, label= "Short")
plt.plot(df2[-500:].loc[df2.entryR == -2].index, df2[-500:]['9-min'][df2.entryR == -2], "v",
        color = "g", markersize = 12, label="Long")
plt.legend(loc=2);
plt.savefig('./static/assets/dualPlot.png')


In [ ]:
bestFit = df2[['time','close', 'market_returns','range_returns', 'trend_returns']]
bestFit.head(30)

* check val in each algo_return, determine greatest val, declear winner
* if declared winner, iterate forward len(X_minutes) in winner col and append new val to list
* check both val at winning location for greatest val, decleare winner
* iterate forward len(X_min) in winning col and append val to list
* repeat, i think..

In [ ]:
count=[]
selectVal = []
greatestVal = []
signal = ''
period = 3

for j, k in bestFit.iterrows():
    count.append(1)    
    if len(count) == period:
        selectVal.append(["time", k['market_returns'], "rangeRet", k['range_returns'], "trendRet", k['trend_returns']])
        if k['range_returns'] > k['trend_returns']:
            signal = -1
            greatestVal.append([ signal, k['range_returns']])
#             selectVal.append([k['range_returns'], signal])
#             np.insert(winnerVal, 1, greatestVal, axis = 1)
        elif k['range_returns'] < k['trend_returns']:
            signal = 1
            greatestVal.append([signal, k['trend_returns']])
#             selectVal.append([k['trend_returns'], signal])
        else:
            k['range_returns'] == k['trend_returns']
            signal = 0
            greatestVal.append([signal, k['trend_returns']])
#             selectVal.append([k['trend_returns'], signal])
        count = []

record=np.concatenate((selectVal,greatestVal), axis=1)
record

        
df3 = pd.DataFrame((record),
                     columns=['time', 'market_returns', 'range','rangeRet', 'trend', 'trendRet', 'winningAlgo', 'winnersRets'])   
df3.drop(['range', 'trend'], axis = 1, inplace=True)
record


In [ ]:
df3['addTrend'] = df3['trendRet'].shift(-1)
df3['addRange'] = df3['rangeRet'].shift(-1)
df3['result'] = np.where(df3['trendRet'] > df3['rangeRet'], df3['addTrend'], np.nan)
df3['result'] = np.where(df3['trendRet'] < df3['rangeRet'], df3['addRange'], df3['result'])
df3['result'] = np.where(df3['trendRet'] == df3['rangeRet'], df3['addRange'], df3['result'])
df3

In [ ]:
initial_capital = float(100000.0)

# Create df positions
positions = pd.DataFrame(index=df2.time.index).fillna(0.0)


# Buy 2 BTC
positions['BTC-PERP'] = 2*df['position']

# Initilize portfolio w value owned
portfolio = positions.multiply(df['close'], axis=0)

# Store diff in shares owned
pos_diff = positions.diff()

# Add 'holdings' to portfolio
portfolio['holdings'] = (positions.multiply(df['close'], axis=0)).sum(axis=1)

# Add 'cash' to portfolio
portfolio['cash'] = initial_capital - (pos_diff.multiply(df['close'], axis=0)).sum(axis=1).cumsum()

# Add 'total' to portfolio
portfolio['total'] = portfolio['cash'] + portfolio['holdings']

# Add 'returns' to portfolio
portfolio['returns'] = portfolio['total'].pct_change()

portfolio['time'] = df['time']
fig = plt.figure(facecolor=(1, 1, 1))
x = portfolio.iloc[-200:]['time']
y = portfolio.iloc[-200:]['total']
plt.xticks(fontsize=22, color="black")
plt.xlabel('Time', color='black',fontsize=22)
plt.yticks(fontsize=22, color='black')
plt.ylabel('Value', color='black',fontsize=22)
plt.plot(values,y)
# plt.savefig('../web/assets/portfolioStandings.png')
plt.show()